In this notebook, we show how to apply the JSR theory to compute the rate of convergence of agents to consensus.
This example is [Example 2.52, P17].

[P17] M. Philippe.
*Path-Complete Methods and Analysis of Constrained Switching Systems*
Doctoral dissertation, UCLouvain, **2017**

In [1]:
using LinearAlgebra
P = [ 1.0  0.0  1.0  2.0
     -1.0  0.0  1.0  2.0
      0.0  1.0  0.0 -3.0
      0.0 -1.0  0.0 -3.0
      0.0  0.0 -2.0  2.0]
for i in 1:4
    P[:, i] /= norm(P[:, i])
end
@show P' * ones(5)
round.(P' * P, digits=16)

P' * ones(5) = [0.0, 0.0, 0.0, 0.0]


4×4 Array{Float64,2}:
 1.0   0.0   0.0   0.0
 0.0   1.0   0.0  -0.0
 0.0   0.0   1.0  -0.0
 0.0  -0.0  -0.0   1.0

In [2]:
using HybridSystems
H1 = [0.2 0.8 0   0   0
      0   0.2 0.8 0   0
      0   0   0.2 0.8 0
      0   0   0   0.2 0.8
      0.2 0   0   0   0.8]
A1 = P' * H1 * P
H2 = [1   0   0   0   0
      0.8 0.2 0   0   0
      0   0   1   0   0
      0   0   0.8 0.2 0
      0   0   0   0   1]
A2 = P' * H2 * P
H3 = [1   0   0   0   0
      0   1   0   0   0
      0   0.8 0.2 0   0
      0   0   0   1   0
      0   0   0   0.2 0.8]
A3 = P' * H3 * P
function automaton(N)
    # See [P17, Figure 2.17] for what automaton(3) should be
    a = LightAutomaton(N)
    # Node i means, H1 was used i-1 steps ago
    add_transition!(a, 1, 1, 1)
    for i in 2:N
        add_transition!(a, i-1, i, 2)
        add_transition!(a, i-1, i, 3)
        add_transition!(a, i, 1, 1)
    end
    return a
end
ss(N) = discreteswitchedsystem([A1, A2, A3], automaton(N))

ss (generic function with 1 method)

In [3]:
using MathOptInterfaceMosek
using JuMP
factory = with_optimizer(MosekOptimizer, QUIET=true);

┌ Info: Recompiling stale cache file /home/blegat/.julia/compiled/v1.0/MathOptInterfaceMosek/sqIeN.ji for MathOptInterfaceMosek [0087ddc6-3964-5e57-817f-9937aefb0357]
└ @ Base loading.jl:1187


# $N = 1$

With $N = 1$, the system is not switched, there is only one matrix and the JSR is approximately $0.7273$.

In [6]:
using SwitchOnSafety
ρ(A1)

0.7273364816948462

In [5]:
using SwitchOnSafety
soslyapb(ss(1), 1; factory=factory, tol=1e-7, verbose=1)

┌ Info: Precompiling SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510]
└ @ Base loading.jl:1189


Lower bound: 0.7273364445083259
Upper bound: 0.7273364878609576
Log-diff   : 5.960464477539063e-8 ≤ 1.0e-7


(0.7273364445083259, 0.7273364878609576)

# $N = 2$

In [4]:
ss2 = ss(2);

We start with CQLF (Common Quadratic Lyapunov Function)

In [5]:
using SwitchOnSafety
soslyapb(ss2, 1; factory=factory, tol=1e-7, verbose=1)

┌ Info: Recompiling stale cache file /home/blegat/.julia/compiled/v1.0/SwitchOnSafety/PA0qe.ji for SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510]
└ @ Base loading.jl:1187


Lower bound: 0.9871583755377715
Upper bound: 0.9871584685372652
Log-diff   : 9.420928959863462e-8 ≤ 1.0e-7


(0.7760559169937989, 0.9871584685372652)

In [7]:
@time seq = sosbuildsequence(ss2, 1, niter=100, l=1, p_0=:Primal)
@time psw = findsmp(seq)

  0.073220 seconds (762.13 k allocations: 48.318 MiB, 35.33% gc time)
  0.032766 seconds (182.71 k allocations: 16.224 MiB, 33.92% gc time)


PSW(0.926231, [3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1])

In [10]:
using SwitchOnSafety
soslyapb(ss2, 2; factory=factory, tol=1e-5, verbose=1)

Lower bound: 0.9270889865323796
Upper bound: 0.927096943835128
Log-diff   : 8.583068847753395e-6 ≤ 1.0e-5


(0.8220050994896435, 0.927096943835128)

In [13]:
@time seq = sosbuildsequence(ss2, 2, niter=100, l=2, p_0=:Primal)
@time psw = findsmp(seq)

  0.916764 seconds (3.84 M allocations: 330.906 MiB, 23.79% gc time)
  0.031345 seconds (181.82 k allocations: 16.196 MiB, 25.22% gc time)


PSW(0.926231, [3, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1])

In [19]:
using SwitchOnSafety
soslyapb(ss2, 3; factory=factory, tol=1e-3, verbose=1)

Lower bound: 0.939646390527466
Upper bound: 0.9404209543616947
Log-diff   : 0.0008239746093749722 ≤ 0.001


(0.8672277705057261, 0.9404209543616947)

In [24]:
@time seq = sosbuildsequence(ss2, 3, niter=45, l=1, p_0=:Primal)
@time psw = findsmp(seq)

  1.778828 seconds (13.32 M allocations: 1.151 GiB, 14.25% gc time)
  0.002662 seconds (25.15 k allocations: 2.366 MiB)


PSW(0.939188, [3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 3, 1])

# $N = 3$

In [18]:
using SwitchOnSafety
soslyapb(ss(3), 1; factory=factory, tol=1e-7, verbose=1)

Lower bound: 1.0872588130791716
Upper bound: 1.087258910287689
Log-diff   : 8.940696726023045e-8 ≤ 1.0e-7


(0.8016932276856896, 1.087258910287689)

# $N = 4$

In [19]:
using SwitchOnSafety
soslyapb(ss(4), 1; factory=factory, tol=1e-7, verbose=1)

Lower bound: 1.1047906239525065
Upper bound: 1.1047906898031612
Log-diff   : 5.960464481702399e-8 ≤ 1.0e-7


(0.7957484559532496, 1.1047906898031612)